###  step1:-  Importing the libraries

In [ ]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", None)

# Import psql to run queries 

import pandasql as psql

### step:- Load data_set

In [ ]:
#Load 
rankdt=pd.read_csv(r"data.csv",header=0)

rankdt_BK=rankdt.copy()

rankdt.head()

### step2:- Information

In [ ]:
# Run info()
rankdt.info()

### step3:- check null values

In [ ]:
# reading null values
rankdt.isnull().sum()

### step4:- Unique values

In [ ]:
rankdt.nunique()

### step5:- Read Columns

In [ ]:
rankdt.columns

In [ ]:
col=['id', 'year', 'institute_type', 'round_no', 'quota', 'pool',
       'institute_short', 'program_name', 'program_duration', 'degree_short',
       'category', 'opening_rank', 'closing_rank', 'is_preparatory']

### step6:- Duplicate Values

In [ ]:
rankdt_dup=rankdt[rankdt.duplicated(keep='last')]
rankdt_dup

In [ ]:
rankdt.drop_duplicates(inplace=True)
rankdt.duplicated().any()

In [ ]:
rankdt["degree_short"].value_counts()


In [ ]:
rankdt["institute_type"].value_counts()

In [ ]:
rankdt["quota"].value_counts()

### Lableencoder for target variables

In [ ]:
# use lableencoder for target variables
from sklearn.preprocessing import LabelEncoder

LE=LabelEncoder()
rankdt['institute_type']=LE.fit_transform(rankdt['institute_type'])
rankdt['quota']=LE.fit_transform(rankdt['quota'])
rankdt['pool']=LE.fit_transform(rankdt['pool'])
rankdt['institute_short']=LE.fit_transform(rankdt['institute_short'])
rankdt['program_name']=LE.fit_transform(rankdt['program_name'])
rankdt['program_duration']=LE.fit_transform(rankdt['program_duration'])
rankdt['degree_short']=LE.fit_transform(rankdt['degree_short'])
rankdt['category']=LE.fit_transform(rankdt['category'])

In [ ]:
rankdt

In [ ]:
#target

closing_rank_count=rankdt.closing_rank.value_counts()
print('Class 0:',closing_rank_count[0])
print('Class 1:',closing_rank_count[1])
print('Proportion:',round(closing_rank_count[0] /closing_rank_count[1] ,2) , ':1')
print('Total Nit records:',len(rankdt))


In [ ]:
rankdt.info()

In [ ]:
rankdt.head()

### step7:-Identify the independent and target variable

In [ ]:
#Identify the independent and target variable

IndepVar=[]
for col in rankdt.columns:
    if col!='is_preparatory':
        IndepVar.append(col)
        
        
TargetVar='is_preparatory'

x=rankdt[IndepVar]
y=rankdt[TargetVar]

In [ ]:
pip install imblearn


### Oversample

In [ ]:
# Random oversampling can be implemented using the RandomOverSampler class

from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy=0.125)

x_over, y_over = oversample.fit_resample(x, y)

print(x_over.shape)
print(y_over.shape)

### step7:-Split the data into train and test(random sampling)

In [ ]:
# Splitting the dataset into train and test 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_over, y_over, test_size = 0.30, random_state = 42)

# Display the shape 

x_train.shape, x_test.shape, y_train.shape, y_test.shape

### step8:-Scaling the features by using MinMaxScaler

In [ ]:
#Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler=MinMaxScaler(feature_range=(0,1))

x_train=mmscaler.fit_transform(x_train)
#x_train[cols]=mmscaler.fit_transform(x_train[cols])
x_train=pd.DataFrame(x_train)

x_test=mmscaler.fit_transform(x_test)
#x_test[cols]=mmscaler.fit_transform(x_test[cols])
x_test=pd.DataFrame(x_test)

In [ ]:
EMResults=pd.read_csv(r"EMResults.csv",header=0)
EMResults.head()

### Compare Classification

In [ ]:
#==Step_02=============================================================================================================>

# Build the Calssification models and compare the results

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Create objects of classification algorithms with default hyper-parameters

ModelLR = LogisticRegression()
ModelDC = DecisionTreeClassifier()
ModelRF = RandomForestClassifier()
ModelET = ExtraTreesClassifier()
ModelKNN = KNeighborsClassifier(n_neighbors=5)
ModelGNB = GaussianNB()
ModelSVM = SVC(probability=True)

# Evalution matrix for all the algorithms

#MM = [ModelLR, ModelDC, ModelRF, ModelET, ModelKNN, ModelGNB, ModelSVM]
MM = [ModelLR, ModelDC, ModelRF, ModelET,ModelKNN, ModelGNB, ModelSVM]
for models in MM:
            
    # Train the model training dataset
    
    models.fit(x_train, y_train)
    
    # Prediction the model with test dataset
    
    y_pred = models.predict(x_test)
    y_pred_prob = models.predict_proba(x_test)
    
    # Print the model name
    
    print('Model Name: ', models)
    
    # confusion matrix in sklearn

    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report

    # actual values

    actual = y_test

    # predicted values

    predicted = y_pred

    # confusion matrix

    matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
    print('Confusion matrix : \n', matrix)

    # outcome values order in sklearn

    tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
    print('Outcome values : \n', tp, fn, fp, tn)

    # classification report for precision, recall f1-score and accuracy

    C_Report = classification_report(actual,predicted,labels=[1,0])

    print('Classification report : \n', C_Report)

    # calculating the metrics

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    # Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
    # A model with a score of +1 is a perfect model and -1 is a poor model

    from math import sqrt

    mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

    print('Accuracy :', round(accuracy*100, 2),'%')
    print('Precision :', round(precision*100, 2),'%')
    print('Recall :', round(sensitivity*100,2), '%')
    print('F1 Score :', f1Score)
    print('Specificity or True Negative Rate :', round(specificity*100,2), '%'  )
    print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
    print('MCC :', MCC)

    # Area under ROC curve 

    from sklearn.metrics import roc_curve, roc_auc_score

    print('roc_auc_score:', round(roc_auc_score(actual, predicted), 3))
    
    # ROC Curve
    
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import roc_curve
    Model_roc_auc = roc_auc_score(actual, predicted)
    fpr, tpr, thresholds = roc_curve(actual, models.predict_proba(x_test)[:,1])
    plt.figure()
    #
    plt.plot(fpr, tpr, label= 'Classification Model' % Model_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()
    print('-----------------------------------------------------------------------------------------------------')
    #----------------------------------------------------------------------------------------------------------
    new_row = {'Model Name' : models,
               'True_Positive': tp,
               'False_Negative': fn, 
               'False_Positive': fp, 
               'True_Negative': tn,
               'Accuracy' : accuracy,
               'Precision' : precision,
               'Recall' : sensitivity,
               'F1 Score' : f1Score,
               'Specificity' : specificity,
               'MCC':MCC,
               'ROC_AUC_Score':roc_auc_score(actual, predicted),
               'Balanced Accuracy':balanced_accuracy}
    EMResults = EMResults.append(new_row, ignore_index=True)
    #----------------------------------------------------------------------------------------------------------
#======================================================================================================================>


In [ ]:
# EMResults
EMResults

### Predict result

In [ ]:
y_predRF=ModelET.predict(x_test)

In [ ]:
Results=pd.DataFrame({'is_preparatory_A':y_test,'is_preparatory_P':y_predRF})

ResultsFinal=rankdt_BK.merge(Results,left_index=True,right_index=True)

ResultsFinal.sample(10)